In [1]:
import numpy as np 
import pygame 
import math
import sys 
from copy import deepcopy 
import time 
import random 
import os
import threading

pygame 2.1.2 (SDL 2.0.18, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [13]:
#Game="A4x4" # "A6x6" "G7x7" "G9x9" "A5x5"
#SIZE = 5
SQUARE_SIZE = 75 

PLAYER1 = 1
PLAYER2 = -1

BLUE = (0,0,255) 
RED = (255,0,0) 
WHITE_COLOR = (255,255,255) 
BLACK_COLOR = (0,0,0) 


In [14]:


class Ataxx():
    def __init__(self, size):
        self.size = size
        self.board_matrix = np.zeros((self.size, self.size)).astype(int)
        self.to_move = PLAYER1
        #self.next = PLAYER2

        self.board_matrix[0][0] = PLAYER1
        self.board_matrix[self.size-1][self.size-1] = PLAYER1
        self.board_matrix[0][self.size-1] = PLAYER2
        self.board_matrix[self.size-1][0] = PLAYER2  


    
    def place(self, x, y): 
        self.board_matrix[x][y] = self.to_move 


    
    def jump(self, x, y, new_x, new_y): 
        self.board_matrix[x][y] = 0
        self.board_matrix[new_x][new_y] = self.to_move

        
    def calc_dist(self, x1, y1, x2, y2):
        return math.sqrt((x1-x2)**2+(y1-y2)**2)



    def play_move(self, x, y, new_x, new_y):

        if new_x < 0 or new_x >= self.size or new_y < 0 or new_y >= self.size:
            return False

        if self.board_matrix[new_x][new_y] != 0:
            return False

        dist = self.calc_dist(x, y, new_x, new_y)

        if dist == 0:
            return False
        
        elif dist < 2:
            self.place(new_x, new_y)
            return True

        elif dist < 3:
            self.jump(x, y, new_x, new_y)
            return True

        else:
            return False
    

    def is_valid_piece(self, x, y):
        if self.board_matrix[x][y] == self.to_move:
            return True
        else:
            return False

    
    def eat(self, x, y):
        for i in range(-1, 2):
            for j in range(-1, 2):
                if x+i < self.size and x+i >=0 and y+j < self.size and y+j >=0: 
                    if self.board_matrix[x+i][y+j] == -self.to_move:
                        self.board_matrix[x+i][y+j] = self.to_move
    
    
    def get_player_pieces(self): 
        pieces = []
        for x in range(self.size):
            for y in range (self.size): 
                if self.board_matrix[x][y] == self.to_move:
                    pieces.append([x,y])
        return pieces  
    
    
        
    def available_moves(self):
        moves = []
        for piece in self.get_player_pieces():
            x = piece[0]
            y = piece[1]
            for i in range(-3, 3):
                for j in range(-3, 3):
                    temp1_board = Ataxx(self.size)
                    temp1_board.board_matrix = self.board_matrix.copy()
                    if temp1_board.play_move(x, y, x+i, y+j):
                        moves.append((temp1_board.board_matrix, (x, y), (x+i, y+j)))
        return moves 

    
    def is_game_over(self):
        if len(self.available_moves()) == 0:
            return True
        return False
    
    
    def piece_counter(self):
        player_1_counter = len(self.get_player_pieces(1))
        player_minus1_counter = len(self.get_player_pieces(-1))
        return player_1_counter, player_minus1_counter
    
    def turn(self):
        """Keep track of the turn by flipping between BLACK and WHITE."""
        self.to_move = -self.to_move
        """
        if self.to_move == PLAYER2:
            self.to_move = PLAYER1
            #self.next = PLAYER2
        else:
            self.to_move = PLAYER2
            #self.next = PLAYER1
        """

    
                
    def draw_pieces(self, screen): 
        for k in range(self.size):
            for l in range(self.size):
                if self.board_matrix[k][l] == PLAYER1: 
                    pygame.draw.circle(screen, RED, (int((l+1)*SQUARE_SIZE - SQUARE_SIZE/2),int((k+1)*SQUARE_SIZE - SQUARE_SIZE/2)), SQUARE_SIZE/3)
                
                elif self.board_matrix[k][l] == PLAYER2: 
                    pygame.draw.circle(screen, BLUE, (int((l+1)*SQUARE_SIZE - SQUARE_SIZE/2),int((k+1)*SQUARE_SIZE - SQUARE_SIZE/2)), SQUARE_SIZE/3)
                
                else: 
                    pygame.draw.circle(screen, BLACK_COLOR, (int((l+1)*SQUARE_SIZE - SQUARE_SIZE/2),int((k+1)*SQUARE_SIZE - SQUARE_SIZE/2)), SQUARE_SIZE/3)
        
        pygame.display.update() 
    
    
    def draw_lines(self, screen): 
        for i in range(self.size -  1):
            pygame.draw.line(screen, WHITE_COLOR, ((i+1)*SQUARE_SIZE,0), ((i+1)*SQUARE_SIZE,SQUARE_SIZE*self.size), 2) 
            pygame.draw.line(screen, WHITE_COLOR, (0,(i+1)*SQUARE_SIZE), (SQUARE_SIZE*self.size,(i+1)*SQUARE_SIZE),2)    
        pygame.display.update() 
    
    def draw_grey_circles(self, screen, x, y):
        for i in range(-2, 3):
            for j in range(-2, 3):
                if x+i >= 0 and x+i < self.size and y+j >= 0 and y+j < self.size:

                    if self.board_matrix[x+i][y+j] == 0:
                        pygame.draw.circle(screen, GREY_COLOR, (int((y+j+1)*SQUARE_SIZE - SQUARE_SIZE/2),int((x+i+1)*SQUARE_SIZE - SQUARE_SIZE/2)), SQUARE_SIZE/3)
                        

    def print_board(self):
       
        #os.system('cls' if os.name == 'nt' else 'clear')
        
        for i in range(self.size): #Imprimir a matriz
            line = ""
            for j in range(self.size):
                if self.board_matrix[i][j] != -1:
                    line += " " + str(self.board_matrix[i][j]) + " "
                else:
                    line += str(self.board_matrix[i][j]) + " "

            print(line)
    
    
    
    



In [15]:
import socket
import random
import time

#Game="A4x4" # "A6x6" "G7x7" "G9x9" "A5x5"


def generate_random_move():
    x = random.randint(0, 9)
    y = random.randint(0, 9)
    return f"MOVE {x},{y}"

def generate_random_move2():
    x = random.randint(0, 9)
    y = random.randint(0, 9)
    x2 = random.randint(0, 9)
    y2 = random.randint(0, 9)
    return f"MOVE {x},{y},{x2},{y2}"

#def random_valid_move(game):
#    available_moves = game.available_moves()
#    move = random.choice(available_moves)
#    return move





def random_valid_move(game, timeout=2):
    result = []

    def target():
        nonlocal result
        try:
            while True:
                i=0
            available_moves = game.available_moves()
            move = random.choice(available_moves)
            result.append(move)
        except Exception as e:
            result.append(e)

    thread = threading.Thread(target=target)
    thread.start()
    thread.join(timeout)

    if thread.is_alive():
        # Function is running longer than the timeout duration
        result.append(TimeoutError("Function execution timed out"))

    if isinstance(result[0], Exception):
        return False  # Raise any exception that occurred inside the thread

    return result[0] if result else None


    

In [18]:
#game = Ataxx(SIZE)

def connect_to_server(host='localhost', port=12345):
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect((host, port))
    
    response = client_socket.recv(1024).decode()
    print(f"Server ResponseINIT: {response}")
    
    Game = response[-4:]
    print("Playing:", Game)
    SIZE = int(Game[-1])
    if Game[0] == "A":
        game = Ataxx(SIZE)
    
    width = SIZE * SQUARE_SIZE 
    height = SIZE * SQUARE_SIZE 
    width = SIZE * SQUARE_SIZE 
    height = SIZE * SQUARE_SIZE 
    screen_size = (width, height) 
    
    
    
    if "1" in response:
        ag=1
    else:
        ag=2
    first=True
    
    
    #game.print_board()
    
    invalid_counter = 0
    
    is_pass = False
    

    screen = pygame.display.set_mode(screen_size)   
    pygame.display.set_caption(f'Agent {ag}')

    game.draw_lines(screen)
    game.draw_pieces(screen)
    time.sleep(10)
    
    while True:
        
        #while True:
        #    print("ok")
        
        pygame.event.pump()
        pygame.display.update()

        if (ag == 1 or not first) and not is_pass:
            if invalid_counter >= 3:
                is_pass = True
                invalid_counter = 0
                game.turn()
            else:
                
                if not game.is_game_over():
                    #print(f"INVALID COUNTER -> {invalid_counter}")
                #move = generate_random_move2()
                    #x1, y1, x2, y2 = map(int, input("Move: ").split())
                    move = random_valid_move(game)
                    if not move:
                        is_pass = True
                        invalid_counter = 0
                        game.turn()
                        continue
                    x1, y1, x2, y2 = move[1][0], move[1][1], move[2][0], move[2][1]
                    #x1, y1, x2, y2 = 0, 0, 1, 1
                    move_msg = f"MOVE {x1},{y1},{x2},{y2}"
                    time.sleep(1)
                    client_socket.send(move_msg.encode())
                    print("Send:",move_msg)

                # Wait for server response
                response = client_socket.recv(1024).decode()
                print(f"Server Response1: {response}")
                if "END" in response: break

                if response == "VALID":
                    game.play_move(x1, y1, x2, y2)
                    game.eat(x2, y2)
                    
                    game.draw_pieces(screen)
                    pygame.display.update() 
                    
                    #game.print_board()
                    game.turn()
                    invalid_counter = 0
                elif response == "INVALID":
                    invalid_counter += 1
                    continue

        first=False
        response = client_socket.recv(1024).decode()
        print(f"Server Response2: {response}")
        
        if response == "YOU":
            game.turn()
            continue
            
        if "END" in response: break
            
        oppx1, oppy1 , oppx2, oppy2 = int(response[5]), int(response[7]), int(response[9]), int(response[11])
        game.play_move(oppx1, oppy1, oppx2, oppy2)
        game.eat(oppx2, oppy2)
        
        game.draw_pieces(screen)
        pygame.display.update() 

        game.print_board()
        game.turn()
        is_pass = False

                
                
        # Add some condition to break the loop, if necessary
        # Example: If server sends a certain message, or after a number of moves
    game_over = True

    while game_over:
        for event in pygame.event.get(): 
            if event.type == pygame.QUIT:
                pygame.quit()
                #sys.exit()

                client_socket.close()
                game_over = False
                
if __name__ == "__main__":
    pygame.init()
    connect_to_server()

Server ResponseINIT: AG1 A6x6
Playing: A6x6
Server Response2: MOVE 5,0,3,2
 1  0  0  0  0 -1 
 0  0  0  0  0  0 
 0  0  0  0  0  0 
 0  0 -1  0  0  0 
 0  0  0  0  0  0 
 0  0  0  0  0  1 
Server Response2: MOVE 0,5,2,5
 1  0  0  0  0  0 
 0  0  0  0  0  0 
 0  0  0  0  0 -1 
 0  0 -1  0  0  0 
 0  0  0  0  0  0 
 0  0  0  0  0  1 
Server Response2: MOVE 2,5,4,5
 1  0  0  0  0  0 
 0  0  0  0  0  0 
 0  0  0  0  0  0 
 0  0 -1  0  0  0 
 0  0  0  0  0 -1 
 0  0  0  0  0 -1 
Server Response2: MOVE 3,2,1,4
 1  0  0  0  0  0 
 0  0  0  0 -1  0 
 0  0  0  0  0  0 
 0  0  0  0  0  0 
 0  0  0  0  0 -1 
 0  0  0  0  0 -1 
Server Response2: MOVE 4,5,3,4
 1  0  0  0  0  0 
 0  0  0  0 -1  0 
 0  0  0  0  0  0 
 0  0  0  0 -1  0 
 0  0  0  0  0 -1 
 0  0  0  0  0 -1 
Server Response2: MOVE 3,4,4,3
 1  0  0  0  0  0 
 0  0  0  0 -1  0 
 0  0  0  0  0  0 
 0  0  0  0 -1  0 
 0  0  0 -1  0 -1 
 0  0  0  0  0 -1 
Server Response2: MOVE 4,5,2,5
 1  0  0  0  0  0 
 0  0  0  0 -1  0 
 0  0  0  0  0 -1